# Libs

In [4]:
from datetime import datetime
import os
import pandas as pd

# Utils 

## Consts

In [2]:
TODAY = datetime.now()
NUMBER_OF_REPOSITORIES=1000
PRIMARY_LANGUANGE = 'java'

ROOT_PATH = os.getcwd().split('\\lab-experimentacao-02')[0].replace('\\', '/')
BASE_URL_GITHUB = 'https://github.com/'
CK_REPO = 'https://github.com/mauricioaniche/ck'

## Functions

In [3]:
def clone_repo(dir_path: str, nameWithOwner: str)-> bool:
  repo_url = f'{BASE_URL_GITHUB}/{nameWithOwner}'
  os.chdir(dir_path)
  return os.system(f'git clone {repo_url}') == 0

    
def execute_ck(repo_name: str, use_jars: str = 'true',max_files_per_partition: int = 0,  variables_and_fields_metrics: str = 'false')-> bool:
  ck_path = f'{ROOT_PATH}/ck/target/ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar'
  project_path = f'{ROOT_PATH}/{repo_name}'
  destiny_path = f'{os.getcwd().replace('\\', '/')}/lab-experimentacao-02/scripts/output/{repo_name}/'
  
  if not os.path.exists(destiny_path):
    os.makedirs(destiny_path)
    
  command_to_run_ck = f'java -jar {ck_path} {project_path} {use_jars} {max_files_per_partition} {variables_and_fields_metrics} {destiny_path}'
  return os.system(command_to_run_ck) == 0
  
    
def delete_repo(repo_name: str = None, *repo_url: str)->bool:
  if not repo_name:
        if repo_url:
            repo_name = repo_url[0].split('/')[-1].replace('.git', '')
        else:
            raise ValueError("repo_name must be provided or calculated from repo_url")
  
  return os.system(f'rmdir /S /Q {repo_name}') == 0

def join_csv(path_files: list):
  if not path_files:
      raise Exception("Lista de arquivos vazia.")
    
  dataframes = []
  
  for path in path_files:
      try:
          df = pd.read_csv(path)
          dataframes.append(df)
      except FileNotFoundError:
          print(f"Arquivo {path} não encontrado.")
          continue
    
  concatenated_df = pd.concat(dataframes, ignore_index=True)
    
  try:
      concatenated_df.to_csv('output.csv', index=False)
      print("Arquivo CSV gerado com sucesso: output.csv")
  except Exception as e:
      print(f"Erro ao escrever arquivo CSV: {e}")


# Script to Clone

In [ ]:
data = pd.read_csv('../dataset/dados_tratados.csv', sep=';')

In [ ]:
destiny_path = ROOT_PATH
repo_teste = 'pabloaugustocm17/rinha-backend-2024-java'

for i in range(2):
    clone_repo(dir_path=destiny_path, nameWithOwner=data['Repositorio'][i])
    execute_ck(repo_name=data['Nome'][i])
    # join_csv()
    delete_repo(repo_name=data['Nome'][i])